In [109]:
from impedance.models.circuits import CustomCircuit
import numpy as np
import matplotlib.pyplot as plt

c = CustomCircuit(initial_guess=[100,1e-3],circuit="p(R_1,C_1)")
z_exp=c.predict(frequencies=np.logspace(-3,3,100))

# plt.plot(np.real(z_exp),-np.imag(z_exp),"o",ms=5)
# plt.gca().text(0.5,0.5,"Experimental Data",transform=plt.gca().transAxes)
# plt.gca().set_aspect("equal")

c2=CustomCircuit(initial_guess=[100,1e-3],circuit="p(R_1,C_1)")
c2.fit(impedance=z_exp,frequencies=np.logspace(-3,3,100))
print(c2)


Circuit string: p(R_1,C_1)
Fit: True

Initial guesses:
    R_1 = 1.00e+02 [Ohm]
    C_1 = 1.00e-03 [F]

Fit parameters:
    R_1 = 1.00e+02  (+/- 0.00e+00) [Ohm]
    C_1 = 1.00e-03  (+/- 0.00e+00) [F]



In [104]:
from scipy.optimize import minimize

omega=2*np.pi*np.logspace(-3,3,100)

import numpy as np
def fun(params,omega,z_exp,weights):
    R=params[0]
    C=params[1]
 
    chi_total=0
    for w,z,wt in zip(omega,z_exp,weights):
        z_model=1/(1/R+1j*C*w)
        # print(z_model)
        chi_real=(np.real(z_model)-np.real(z))**2/wt**2
        chi_imag=(np.imag(z_model)-np.imag(z))**2/wt**2
        # print(chi_real)
        chi_total+=(chi_real+chi_imag)

    return chi_total

# x=np.linspace(1,100,1000)
# plt.plot(x,fun(x))
omega=np.logspace(-3,3,100)
wt=np.abs(z_exp)
result=minimize(fun,(10,1e-3),args=(omega,z_exp,wt),method="Nelder-Mead")
print(result)


       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 2.921720203512872e-11
             x: [ 1.000e+02  6.283e-03]
           nit: 129
          nfev: 237
 final_simplex: (array([[ 1.000e+02,  6.283e-03],
                       [ 1.000e+02,  6.283e-03],
                       [ 1.000e+02,  6.283e-03]]), array([ 2.922e-11,  8.407e-11,  1.029e-10]))
